Data from https://github.com/M3IT/COVID-19_Data/raw/master/Data/COVID_AU_state.csv
and saved in `aust_covid` folder.

In [ ]:
import pandas as pd
pd.options.plotting.backend = "plotly"
import plotly.express as px
from datetime import datetime

from summer2 import CompartmentalModel
from summer2.parameters import Parameter, DerivedOutput
from summer2.utils import ref_times_to_dti

In [ ]:
state_data = pd.read_csv(
    "./aust_covid/COVID_AU_state.csv", 
    index_col="date",
)
state_data.index = pd.to_datetime(state_data.index)
ref_date = datetime(2019, 12, 31)

In [ ]:
model = CompartmentalModel(
    times=(600, 800),
    compartments=(
        "susceptible",
        "infectious",
        "recovered",
    ),
    infectious_compartments=("infectious",),
)
model.set_initial_population(
    {
        "susceptible": 2.6e7,
        "infectious": 1.0,
    }
)
model.add_infection_frequency_flow(
    "infection",
    Parameter("contact_rate"),
    "susceptible",
    "infectious",
)
model.add_transition_flow(
    "recovery",
    1.0 / Parameter("infectious_period"),
    "infectious",
    "recovered",
)

model.request_output_for_flow(
    "onset",
    "infection",
)
model.request_function_output(
    "notifications",
    func=DerivedOutput("onset") * Parameter("cdr"),
)

In [ ]:
parameters = {
    "contact_rate": 0.3,
    "infectious_period": 5.0,
    "cdr": 0.3,
}
model.run(parameters=parameters)

In [ ]:
outputs = model.get_derived_outputs_df()
outputs.index = ref_times_to_dti(
    ref_date, 
    outputs.index,
)

In [ ]:
comparison_df = pd.concat(
    (
        state_data["confirmed"], 
        outputs["notifications"],
    ), 
    axis=1,
)

In [ ]:
comparison_df.plot()